In [1]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [2]:
series = pd.read_csv('..\clean\series_after_cleaning.csv')

In [3]:
series["word_cloud"]=series["description"]+""+series["genre"]
series["title"]=series["title"].str.lower()

In [4]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
series["word_cloud"] = series["word_cloud"].fillna('')  
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(series['word_cloud'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(1021, 5462)

In [5]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
indices = pd.Series(series.index, index=series['title']).drop_duplicates()


In [7]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return series['title'].iloc[movie_indices]

In [8]:
get_recommendations('breaking bad')


535    the fall of the house of usher
40                            persona
558                             weeds
965                         the order
433                            castle
851                            colony
807                       the undoing
369                  olive kitteridge
625                   defending jacob
704                    the peripheral
Name: title, dtype: object